In [1]:
from keras.applications import vgg16
from keras.preprocessing import image
from quiver_engine.server import launch
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Input
from keras import backend as K
from keras import optimizers
import os, glob
import numpy as np
import cv2
from matplotlib import pyplot as plt

Using TensorFlow backend.


## Model생성
- 기존 VGG16 모델에서 fully connected부분을 제외하고 가져옴
- Flatten함수-> 추가하여 fully connected할 때 곱할 개수 구함
- Dense 함수 -> shape이 (None, 512)인 layer 생성, classification layer(2&5) 생성

In [2]:
# create the base pre-trained model
base_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(120,120,3))
x = base_model.output
x = Flatten(name='flatten')(x) # fully connected할 때 곱할 값을 구함

#x = Dense(2048, activation='relu', name='dense_1')(x)
x = Dense(512, activation='relu', name='dense_2')(x)
#predictions = Dense(5, activation='softmax', name='prediction')(x)
predictions = Dense(2, activation='softmax', name='prediction')(x)

model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False

In [3]:
#train data
#X_train = np.load('cd_img.npy')/255.0
#Y_train = np.load('cd_label.npy')
X=np.zeros((60,120,120,3))
Y=np.zeros((60,2))
X_train = np.load('cd_img2.npy')/255.0
X[0:30] = X_train[0:30]
X[30:60] = X_train[12500:12530]
Y_train = np.load('cd_label2.npy')
Y[0:30] = Y_train[0:30]
Y[30:60] = Y_train[12500:12530]

In [4]:
#test data
X_test=np.zeros((60,120,120,3))
Y_test=np.zeros((60,2))
X_test[0:30] = X_train[30:60]
X_test[30:60] = X_train[12530:12560]
Y_test[0:30] = Y_train[30:60]
Y_test[30:60] = Y_train[12530:12560]

In [5]:
#Randomly shuffle data and labels from different files in the same order
idx = np.random.permutation(len(X))
X_s,Y_s = X[idx], Y[idx]
X_t,Y_t = X_test[idx], Y_test[idx]

- loss : categorical_crossentropy
- optimizer : Adam(Learning Rate : 0.0001)

In [6]:
#train
model.compile(loss = 'categorical_crossentropy', optimizer = optimizers.Adam(lr=0.0001),metrics=["accuracy"])

In [7]:
history = model.fit(X_s,Y_s,epochs=50,batch_size=30, validation_split=0.1)

Train on 54 samples, validate on 6 samples
Epoch 1/50
54/54 [==============================] - 2s 30ms/step - loss: 0.7945 - acc: 0.4074 - val_loss: 0.5229 - val_acc: 1.0000
Epoch 2/50
54/54 [==============================] - 0s 1ms/step - loss: 0.5955 - acc: 0.7037 - val_loss: 0.4101 - val_acc: 0.8333
Epoch 3/50
54/54 [==============================] - 0s 1ms/step - loss: 0.4481 - acc: 0.8889 - val_loss: 0.3759 - val_acc: 1.0000
Epoch 4/50
54/54 [==============================] - 0s 1ms/step - loss: 0.3382 - acc: 0.9630 - val_loss: 0.3763 - val_acc: 1.0000
Epoch 5/50
54/54 [==============================] - 0s 1ms/step - loss: 0.2785 - acc: 0.9815 - val_loss: 0.3473 - val_acc: 1.0000
Epoch 6/50
54/54 [==============================] - 0s 1ms/step - loss: 0.2250 - acc: 0.9815 - val_loss: 0.2782 - val_acc: 1.0000
Epoch 7/50
54/54 [==============================] - 0s 1ms/step - loss: 0.1772 - acc: 1.0000 - val_loss: 0.2486 - val_acc: 1.0000
Epoch 8/50
54/54 [============================

In [8]:
score = model.evaluate(X_t, Y_t,batch_size=30)

60/60 [==============================] - 0s 1ms/step


In [9]:
print(score[0])
print(score[1])

0.438268661499
0.84999999404


In [ ]:
plt.plot(history.history['loss'], 'r-')
plt.plot(history.history['val_loss'], 'b-.')
plt.legend(['loss', 'val_loss'], loc=1)
plt.show()